In [ ]:
import os
import json
import torch
import faiss
import logging
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from Libraries import A0_MyUtils as A0, A1_TextProcess as A1, A2_PdfProcess as A2
from Libraries import B1_ExtractData as B1, B2_MergeData as B2, B3_GetStructures as B3
from Libraries import B4_ChunkMaster as B4, B5_ChunkFlex as B5, B6_ChunkFixed as B6
from Libraries import C1_CreateSchema as C1, C2_Embedding as C2, C3_CheckConstruct as C3
from Libraries import D0_FaissConvert as D0, D1_Search as D1, D2_Rerank as D2, D3_Respond as D3
from Config import Widgets, Configs

In [ ]:
widgets_list = Widgets.create_name_form()

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
force_download = True

In [ ]:
config = Configs.WidgetValues(widgets_list)

data_foler = config["data_folder"]
dcmt_path = config["dcmt_path"]
base_folder = config["base_folder"]
base_path = config["base_path"]
extracted_path = config["extracted_path"]
merged_path = config["merged_path"]
struct_path = config["struct_path"]
chunks_base = config["chunks_base"]
chunks_segment = config["chunks_segment"]
schema_ex_path = config["schema_ex_path"]
embedding_path = config["embedding_path"]
torch_path = config["torch_path"]
faiss_path = config["faiss_path"]
mapping_path = config["mapping_path"]
mapping_data = config["mapping_data"]

FILE_TYPE = config["FILE_TYPE"]
DATA_KEY = config["DATA_KEY"]
EMBE_KEY = config["EMBE_KEY"]
SWITCH = config["SWITCH"]
EMBEDD_MODEL = config["EMBEDD_MODEL"]
SEARCH_EGINE = config["SEARCH_EGINE"]
RERANK_MODEL = config["RERANK_MODEL"]
RESPON_MODEL = config["RESPON_MODEL"]
MERGE = config["MERGE"]
API_KEY = config["API_KEY"]

WORD_LIMIT = config["WORD_LIMIT"]
LEVEL_INPUT = config["LEVEL_INPUT"]
LEVEL_VALUES = config["LEVEL_VALUES"]

Contents = LEVEL_VALUES[-1] if LEVEL_VALUES else None

SEARCH_ENGINE = faiss.IndexFlatIP

In [ ]:
print("CUDA supported:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("Current GPU name:", torch.cuda.get_device_name(0))
    print("CUDA device capability:", torch.cuda.get_device_capability(0))
    print("CUDA version (PyTorch):", torch.version.cuda)
    print("cuDNN version:", torch.backends.cudnn.version())
else:
    print("CUDA not available.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cached_path = "../Models"

def load_auto_model(path_or_name, device):
    try:
        tokenizer = AutoTokenizer.from_pretrained(path_or_name)
        model = AutoModel.from_pretrained(path_or_name).to(device)
        return tokenizer, model
    except (OSError, FileNotFoundError) as e:
        print("❌ Model files missing:", e)
        return None, None
    except RuntimeError as e:
        print("⚠️ GPU issue, fallback to CPU:", e)
        tokenizer = AutoTokenizer.from_pretrained(path_or_name)
        model = AutoModel.from_pretrained(path_or_name).to("cpu")
        return tokenizer, model
    except Exception as e:
        print("❌ Unexpected error:", e)
        raise

def load_sentence_model(path_or_name, device):
    try:
        return SentenceTransformer(path_or_name, device=str(device))
    except (OSError, FileNotFoundError) as e:
        print("❌ Model files missing:", e)
        return None
    except RuntimeError as e:
        print("⚠️ GPU issue, fallback to CPU:", e)
        return SentenceTransformer(path_or_name, device="cpu")
    except Exception as e:
        print("❌ Unexpected error:", e)
        raise


# ================= Main logic =================
if SWITCH == "Auto Model":
    if os.path.exists(cached_path):
        tokenizer, model = load_auto_model(cached_path, device)
        print(f"ℹ️ Auto Model: {cached_path}")
        if model is None:
            tokenizer, model = load_auto_model(EMBEDD_MODEL, device)
    else:
        print(f"ℹ️ Auto Model: {EMBEDD_MODEL}")
        tokenizer, model = load_auto_model(EMBEDD_MODEL, device)

elif SWITCH == "Sentence Transformer":
    if os.path.exists(cached_path):
        model = load_sentence_model(cached_path, device)
        print(f"ℹ️ Sentece Transformer: {cached_path}")

        if model is None:
            model = load_sentence_model(EMBEDD_MODEL, device)
    else:
        print(f"ℹ️ Sentece Transformer: {EMBEDD_MODEL}")
        model = load_sentence_model(EMBEDD_MODEL, device)

print(f"✅ Using: {device}")